In [4]:
from openai import OpenAI
import openai
import pandas as pd
import requests

In [6]:
client = OpenAI(
    api_key="token-vulcan",  # 输入你的 API Key
    base_url=
)




def qa_base(messages):
    completion = client.chat.completions.create(
        model="Qwen2-5-14B",
        messages=messages,
        logprobs=False,

        # stream=True  # 开启流式输出,
    )
    return completion.choices[0].message.content
    # if compile.status_code == 200:
    #     result = completion[0].choices[0].delta.content
    #     return result
    # else:
    #     print(f"请求失败，状态码: {completion.status_code}")
    #     return ""

user_input = "中国的首都是哪里"
input = [{"role": "user", "content": user_input},]
qa_base(input)

'中国的首都是北京。'

In [3]:
text = "甲状腺形态大小正常，边界清晰，实质回声不均质，CDFI:甲状腺实质内未探及异常血流信号。甲状腺左侧叶可见多个结节回声，较大者大小约14.0×9.3mm，甲状腺右侧叶可见多个结节回声，较大者大小约9.0×6.8mm。结构：囊实性。回声：混合回声。方位：水平位。边缘：光整。内部有无强回声无局灶性强回声。血供：无血供。甲状腺左侧叶实质内可探及多个无回声较大者直径约2.6mm其内可见点状强回声甲状腺右侧叶实质内可探及多个无回声，较大者直径约3.6mm,其内可见点状强回声+甲状腺实质弥漫性改变，请结合甲功及相关抗体检查&amp;&amp;甲状腺双侧叶囊实性结节，C-TIRADS分类：3类：&amp;&amp;甲状腺双侧叶胶质囊肿C-TIRADS分类：2类：&amp;&amp;恶性风险评估3类：可能良性,恶性可能＜2%。"
discription,conclusion = text.split("+")
print(discription)
print(conclusion)

甲状腺形态大小正常，边界清晰，实质回声不均质，CDFI:甲状腺实质内未探及异常血流信号。甲状腺左侧叶可见多个结节回声，较大者大小约14.0×9.3mm，甲状腺右侧叶可见多个结节回声，较大者大小约9.0×6.8mm。结构：囊实性。回声：混合回声。方位：水平位。边缘：光整。内部有无强回声无局灶性强回声。血供：无血供。甲状腺左侧叶实质内可探及多个无回声较大者直径约2.6mm其内可见点状强回声甲状腺右侧叶实质内可探及多个无回声，较大者直径约3.6mm,其内可见点状强回声
甲状腺实质弥漫性改变，请结合甲功及相关抗体检查&amp;&amp;甲状腺双侧叶囊实性结节，C-TIRADS分类：3类：&amp;&amp;甲状腺双侧叶胶质囊肿C-TIRADS分类：2类：&amp;&amp;恶性风险评估3类：可能良性,恶性可能＜2%。


In [ ]:
file_path = "./字段文档_简版.xlsx"
df = pd.read_excel(file_path,keep_default_na=False)

## 乳腺

In [11]:
prompt = """
任务说明：

您是一个医疗数据提取助手，专门用于解析乳腺超声报告。您的任务是从给定的报告文本中提取以下结构化信息，并将其填写在指定的单一垂直表格中。请确保信息的准确性和完整性，特别是在提取结节描述和结论部分。

需要提取的信息类别：
1.乳腺结构正常性：
•乳腺层次结构是否清晰。
•腺体排列是否正常。
2.乳腺结节的结构化描述：
•结节所在侧别（左或右）。
•结节位置（如“6点钟”）。
•结节大小（长轴和短轴，单位为厘米）。
•结节边界描述（如“尚清”）。
•结节形态描述（如“规则”）。
•是否见明显血流信号。
•结节的疑似诊断（如“纤维腺瘤？”）。
3.乳腺低回声区的描述：
•低回声区所在侧别。
•低回声区位置。
•低回声区大小（长轴、宽轴、高轴，单位为厘米）。
•低回声区边界描述。
•低回声区形态描述。
•是否见明显血流信号。
•低回声区的疑似诊断（如“腺病？”）。
4.BIRADS分级描述：
•每个结节或低回声区对应的BIRADS分类（如“BIRADS III类”）。
5.报告结论的结构化描述：
•总结报告中的结论部分，包括任何建议或进一步评估的建议。

表格格式要求：

将所有提取的信息按照以下垂直排列的表格格式填写。每个字段在表格的第一列，相关值在第二列。对于存在多个结节或低回声区的情况，使用编号区分每个结节或低回声区的信息。

字段值
报告信息
report_id[填写报告唯一标识，例如1]
patient_id[填写患者唯一标识，例如1001]
exam_date[填写检查日期，例如2024-12-17]
layer_structure[填写乳腺层次结构描述，例如“层次结构清晰”或“层次结构不清”]
gland_arrangement[填写腺体排列描述，例如“腺体排列尚可”或“腺体排列异常”]
axillary_findings[填写双侧腋窝及锁骨下淋巴结探查结果，例如“双侧腋窝及锁骨下未见明显肿大的淋巴结回声”]
supraclavicular_findings[填写双侧锁骨上淋巴结探查结果，例如“双侧锁骨上未见明显异常占位回声”]
CDFI_findings[填写彩色多普勒血流成像结果，例如“未见明显血流信号”或“见异常血流信号”]
乳腺结构是否正常[填写“正常”或“不正常”，根据layer_structure和gland_arrangement的描述]
左侧乳腺信息
side左
duct_dilation[填写TRUE/FALSE，是否有导管扩张，根据报告内容]
duct_dilation_details[填写导管扩张的具体描述，如“局限性增宽，内径约0.23cm”或留空]
blood_flow_present[填写TRUE/FALSE，是否存在异常血流信号]
blood_flow_details[填写血流信号的具体描述，如“未见明显血流信号”或具体血流描述]
nodules_present[填写TRUE/FALSE，是否存在结节]
nodules_count[填写结节数量，如2]
左侧乳腺结节 1
nodule_id1
location6点钟
distance_from_nipple_cm[填写距乳头距离，若报告中未提及则填写NULL]
size_x_cm1.2
size_y_cm0.6
boundary尚清
shape规则
blood_flowFALSE
blood_flow_details未见明显血流信号
左侧乳腺结节 2
nodule_id2
location纤维腺瘤（疑）
distance_from_nipple_cmNULL
size_x_cm1.2
size_y_cm0.6
boundary尚清
shape规则
blood_flowFALSE
blood_flow_details未见明显血流信号
右侧乳腺信息
side右
duct_dilation[填写TRUE/FALSE，是否有导管扩张，若无则填写FALSE]
duct_dilation_details[填写导管扩张的具体描述，若无则留空]
blood_flow_present[填写TRUE/FALSE，是否存在异常血流信号]
blood_flow_details[填写血流信号的具体描述，如“未见明显血流信号”或具体血流描述]
nodules_present[填写TRUE/FALSE，是否存在结节]
nodules_count[填写结节数量，如3]
右侧乳腺结节 1
nodule_id3
location12点钟
distance_from_nipple_cmNULL
size_x_cm0.8
size_y_cm0.3
boundary清
shape规则
blood_flowFALSE
blood_flow_details未见明显血流信号
右侧乳腺结节 2
nodule_id4
location外上象限
distance_from_nipple_cmNULL
size_x_cm4.4
size_y_cm1.1
boundary尚清
shape欠规则
blood_flowFALSE
blood_flow_details未见明显血流信号
右侧乳腺结节 3
nodule_id5
location增生结节（疑）
distance_from_nipple_cmNULL
size_x_cm0.8
size_y_cm0.3
boundary清
shape规则
blood_flowFALSE
blood_flow_details未见明显血流信号
结论BIRADS III类，建议定期随访观察。

示例输入：

“左侧乳腺腺体6点见一低回声结节，大小约1.2cm×0.6cm，界尚清，形态规则，未见明显血流信号，余腺体结构尚清晰，回声尚均匀，右侧乳腺腺体12点探及一低回声结节，大小约0.8cm×0.3cm，界清，形态规则，未见明显血流信号，腺体外上象限见一低回声区范围约4.4cm×1.1cm×3.5cm，界尚清，形态欠规则，未见明显血流信号，余腺体结构尚清晰，回声尚均匀，双侧腋窝及锁骨下未见明显肿大的淋巴结回声+右侧乳腺外上象限低回声区（腺病？）(BI-RADS3类）；右侧乳腺低回声结节（增生结节？）(BI-RADS3类）；左侧乳腺低回声结节（纤维腺瘤？）(BI-RADS3类）”

预期输出：

请将以下报告内容填充到上述表格中,字段和值用#号链接：

字段值
报告信息
report_id1
patient_id1001
exam_date2024-12-17
layer_structure层次结构清晰
gland_arrangement腺体排列尚可
axillary_findings双侧腋窝及锁骨下未见明显肿大的淋巴结回声
supraclavicular_findings双侧锁骨上未见明显异常占位回声
CDFI_findings未见明显血流信号
乳腺结构是否正常正常
左侧乳腺信息
side左
duct_dilationFALSE
duct_dilation_details
blood_flow_presentFALSE
blood_flow_details未见明显血流信号
nodules_presentTRUE
nodules_count2
左侧乳腺结节 1
nodule_id1
location6点钟
distance_from_nipple_cmNULL
size_x_cm1.2
size_y_cm0.6
boundary尚清
shape规则
blood_flowFALSE
blood_flow_details未见明显血流信号
左侧乳腺结节 2
nodule_id2
location纤维腺瘤（疑）
distance_from_nipple_cmNULL
size_x_cm1.2
size_y_cm0.6
boundary尚清
shape规则
blood_flowFALSE
blood_flow_details未见明显血流信号
右侧乳腺信息
side右
duct_dilationFALSE
duct_dilation_details
blood_flow_presentFALSE
blood_flow_details未见明显血流信号
nodules_presentTRUE
nodules_count3
右侧乳腺结节 1
nodule_id3
location12点钟
distance_from_nipple_cmNULL
size_x_cm0.8
size_y_cm0.3
boundary清
shape规则
blood_flowFALSE
blood_flow_details未见明显血流信号
右侧乳腺结节 2
nodule_id4
location外上象限
distance_from_nipple_cmNULL
size_x_cm4.4
size_y_cm1.1
boundary尚清
shape欠规则
blood_flowFALSE
blood_flow_details未见明显血流信号
右侧乳腺结节 3
nodule_id5
location增生结节（疑）
distance_from_nipple_cmNULL
size_x_cm0.8
size_y_cm0.3
boundary清
shape规则
blood_flowFALSE
blood_flow_details未见明显血流信号
结论BIRADS III类，建议定期随访观察。

"""

In [12]:
print(len(prompt))

3926


In [13]:
input = [{"role": "user", "content": prompt},]
output_ = qa_base(input)

In [14]:
print(output_)

根据提供的示例输入内容，我将按照指定的格式提取并组织信息，形成结构化的垂直表格。以下是根据提供的报告内容填充的信息：

字段值
report_id#1
patient_id#1001
exam_date#2024-12-17
layer_structure#层次结构清晰
gland_arrangement#腺体排列尚可
axillary_findings#双侧腋窝及锁骨下未见明显肿大的淋巴结回声
supraclavicular_findings#双侧锁骨上未见明显异常占位回声
CDFI_findings#未见明显血流信号
乳腺结构是否正常#正常
左侧乳腺信息
side#左
duct_dilation#FALSE
duct_dilation_details#
blood_flow_present#FALSE
blood_flow_details#未见明显血流信号
nodules_present#TRUE
nodules_count#2
左侧乳腺结节 1
nodule_id#1
location#6点钟
distance_from_nipple_cm#NULL
size_x_cm#1.2
size_y_cm#0.6
boundary#尚清
shape#规则
blood_flow#FALSE
blood_flow_details#未见明显血流信号
左侧乳腺结节 2
nodule_id#2
location#纤维腺瘤（疑）
distance_from_nipple_cm#NULL
size_x_cm#1.2
size_y_cm#0.6
boundary#尚清
shape#规则
blood_flow#FALSE
blood_flow_details#未见明显血流信号
右侧乳腺信息
side#右
duct_dilation#FALSE
duct_dilation_details#
blood_flow_present#FALSE
blood_flow_details#未见明显血流信号
nodules_present#TRUE
nodules_count#3
右侧乳腺结节 1
nodule_id#3
location#12点钟
distance_from_nipple_cm#NULL
size_x_cm#0.8
size_y_cm#0.3
boundary#清
shape#规则
